In [2]:
import os
import glob

In [3]:
import numpy as np
from matplotlib.pyplot import *

In [4]:
import TimeTagger as TT
from TimeTagger import Flim, TimeTagStream, createTimeTaggerVirtual, EventGenerator, Countrate,ConstantFractionDiscriminator, DelayedChannel

In [25]:
#from tqdm.notebook import tqdm
from fastprogress.fastprogress import progress_bar,master_bar
import tifffile as tf

In [6]:
filename = 'C:\\Users\\lociuser\\Documents\\UserData\\Jenu\\BB_data\\CYCLE_000001_RAWDATA_000034'

In [7]:
tagformat = np.dtype([
    # TimeTag = 0, Error = 1, OverflowBegin = 2, OverflowEnd = 3, MissedEvents = 4
    ('type', 'u1'),
    # ('reserved',      'i1'),
    ('missed_events', '<u2'),
    ('channel', '<i4'),
    ('time', '<i8'),], align=True)

In [27]:
foldername = r'D:\Data\BrianBurkel\TSeries-ms499_NADH_FLIM_780_230620-001'
filelist = glob.glob(os.path.join(foldername,'CYCLE_*_RAWDATA_*'))
len(filelist) # 46 GB

77

'CYCLE_000001_RAWDATA_000034'

In [29]:
mb = master_bar(filelist)
for filename in mb:

    with open(filename,'rb') as fid:
        data_ = np.frombuffer(fid.read(), dtype=tagformat)

    tm = (data_[-1]['time'] - data_[0]['time'])/1e12
    print(f'{os.path.split(filename)[-1]} || {tm:.2f} secs')
    print('|| NONTAGGED' ,len(np.where(data_['type']!=0)[0]),end='')
    print('|| MISSED' ,len(np.where(data_['missed_events']!=0)[0]),end='')
    print('|| PIXELS',len(np.where(data_['channel']==2)[0]),end='')
    print('|| CH1',len(np.where(data_['channel']==1013)[0]),end='')
    print('|| CH2',len(np.where(data_['channel']==1014)[0]),end='\n')
        
        
    sdts=[]
    current_pixel=0
    arr1=[]#np.zeros(512*512)
    arr2=[]#np.zeros(512*512)
    frm_pixel=[0]
    
    for ix,photon_event in progress_bar(enumerate(data_),total=len(data_), parent=mb):
        if current_pixel==0:
            times1 = []
            times2 = []
        if photon_event['channel']==3:
            prev_event=None
            prev_step=1
            while not (prev_event):
                prev_ = data_[ix-prev_step]
                if (prev_['channel']==1013):
                    prev_event=prev_    
                    #px1.append((photon_event,prev_))   
                    times1.append(prev_['time']- photon_event['time'])
                if (prev_['channel']==1014):        
                    prev_event=prev_    
                    #px2.append((photon_event,prev_))
                    times2.append(prev_['time']- photon_event['time'])
                prev_step+=1
        if photon_event['channel']==2:
            current_pixel+=1
            if times1:
                arr1.append(times1)
            else:
                arr1.append([-12501])
            if times2:
                arr2.append(times2)
            else:
                arr2.append([-12501])
            times1 = []
            times2 = []
        #frm = current_pixel/(512*512)
        if (current_pixel%(512*512)==0)&(current_pixel>frm_pixel[-1]):        
            #print(ix,f'\t {ix/len(data_):.2f}',photon_event, current_pixel)
            sdt1=np.zeros([512*512,256])
            sdt2=np.zeros([512*512,256])
            bin1 = int(12500/256)
            for a1x,a in enumerate(arr1):
                if a:
                    tsx=[a1//bin1 for a1 in a if (a1//bin1)>-256]
                    sdt1[a1x,tsx]+=1
            for a1x,a in enumerate(arr2):
                if a:
                    tsx=[a1//bin1 for a1 in a if (a1//bin1)>-256]
                    sdt2[a1x,tsx]+=1

            arr1=[]#np.zeros(512*512)
            arr2=[]#np.zeros(512*512)
            sdts.append((sdt1,sdt2))

            frm_pixel.append(current_pixel)
            
    for ix,(sdt1x,sdt2x) in enumerate(sdts):
        #print(sdt1x.sum(),sdt2x.sum())

        sdt1a = sdt1.reshape([512,512,256]).transpose([2,0,1])
        sdt2a = sdt2.reshape([512,512,256]).transpose([2,0,1])

        tf.imwrite(f'{filename}_ch1_f{ix:>02}.tif',sdt1a.astype(np.uint16))
        tf.imwrite(f'{filename}_ch2_f{ix:>02}.tif',sdt2a.astype(np.uint16))
        

CYCLE_000001_RAWDATA_000034 || 39.20 secs
|| NONTAGGED 0|| MISSED 0|| PIXELS 8388716|| CH1 6627869|| CH2 5039643CYCLE_000001_RAWDATA_000035 || 38.52 secs
|| NONTAGGED 0|| MISSED 0|| PIXELS 8390121|| CH1 5265453|| CH2 4506213CYCLE_000001_RAWDATA_000036 || 37.82 secs
|| NONTAGGED 0|| MISSED 0|| PIXELS 8394626|| CH1 4052790|| CH2 3871444CYCLE_000001_RAWDATA_000037 || 37.33 secs
|| NONTAGGED 0|| MISSED 0|| PIXELS 8390260|| CH1 3488869|| CH2 3452072CYCLE_000002_RAWDATA_000038 || 39.10 secs
|| NONTAGGED 0|| MISSED 0|| PIXELS 8389595|| CH1 6352377|| CH2 4936084CYCLE_000002_RAWDATA_000039 || 38.53 secs
|| NONTAGGED 0|| MISSED 0|| PIXELS 8392207|| CH1 5312333|| CH2 4426089CYCLE_000002_RAWDATA_000040 || 37.81 secs
|| NONTAGGED 0|| MISSED 0|| PIXELS 8391641|| CH1 4127610|| CH2 3787095CYCLE_000002_RAWDATA_000041 || 37.38 secs
|| NONTAGGED 0|| MISSED 0|| PIXELS 8394189|| CH1 3553641|| CH2 3376669CYCLE_000003_RAWDATA_000042 || 39.03 secs
|| NONTAGGED 0|| MISSED 0|| PIXELS 8390369|| CH1 6271166|| CH2